In [482]:
#import des fichiers et load des ressources
import pandas as pd
import numpy as np

data = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
pd.options.display.max_columns = data.shape[1]

In [483]:
data.sample(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
2857,27231,2016,SPS-District K-12,K-12 School,Loyal Heights Elementary,2511 N.W. 80th St.,Seattle,WA,98117.0,4443801580,6,BALLARD,47.68644,-122.38905,1932,1.0,2,44753,0,44753,K-12 School,K-12 School,46545.0,NaN,NaN,NaN,NaN,NaN,96.0,35.200001,39.099998,62.099998,67.199997,1638691.625,1817756.125,0.0,163794.7031,558868.0,10798.24121,1079824.0,True,NaN,Error - Correct Default Data,NaN,61.25,1.37


In [484]:
#variables cohérentes
cols = ['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName','Neighborhood','YearBuilt','NumberofBuildings','NumberofFloors',
        'PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'ENERGYSTARScore',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
        'Electricity(kBtu)','NaturalGas(kBtu)','TotalGHGEmissions', 'GHGEmissionsIntensity']
data = data[cols]

In [485]:
data['BuildingType'].value_counts()

NonResidential          1460
Multifamily LR (1-4)    1018
Multifamily MR (5-9)     580
Multifamily HR (10+)     110
SPS-District K-12         98
Nonresidential COS        85
Campus                    24
Nonresidential WA          1
Name: BuildingType, dtype: int64

In [486]:
data_length_before = data.shape[0]
data['BuildingType'].replace({'Nonresidential COS': 'NonResidential',
                              'Nonresidential WA': 'NonResidential'}, inplace=True)
data = data.loc[data['BuildingType'] == 'NonResidential']

print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1830


In [487]:
#valeurs manquantes
data.isna().sum()

OSEBuildingID               0
DataYear                    0
BuildingType                0
PrimaryPropertyType         0
PropertyName                0
Neighborhood                0
YearBuilt                   0
NumberofBuildings           2
NumberofFloors              0
PropertyGFATotal            0
PropertyGFAParking          0
PropertyGFABuilding(s)      0
LargestPropertyUseType      6
ENERGYSTARScore           540
SiteEnergyUseWN(kBtu)       3
SteamUse(kBtu)              2
Electricity(kBtu)           2
NaturalGas(kBtu)            2
TotalGHGEmissions           2
GHGEmissionsIntensity       2
dtype: int64

In [488]:
#drop toutes lignes qui ont des valeurs nulles sauf EnergyScore
data_length_before = data.shape[0]
cols = ['LargestPropertyUseType', 'SiteEnergyUseWN(kBtu)', 'GHGEmissionsIntensity', 'TotalGHGEmissions', 'NaturalGas(kBtu)', 'Electricity(kBtu)', 'SteamUse(kBtu)', 'NumberofBuildings']
data.dropna(subset=cols, inplace=True)
data.isna().sum()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  7


In [489]:
data.describe()

,OSEBuildingID,DataYear,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity
count,1539.000000,1539.0,1539.000000,1539.000000,1539.000000,1.539000e+03,1539.000000,1.539000e+03,1003.000000,1.539000e+03,1.539000e+03,1.539000e+03,1.539000e+03,1539.000000,1539.000000
mean,16609.779727,2016.0,1961.307342,1.035737,4.283951,1.120106e+05,13829.220273,9.818137e+04,63.790628,7.761522e+06,4.967574e+05,5.435044e+06,1.672264e+06,165.047498,1.609890
std,13858.481096,0.0,32.888052,0.569622,6.796834,1.918960e+05,43730.728115,1.685664e+05,28.923639,1.884349e+07,5.314667e+06,1.305847e+07,5.513280e+06,571.657895,2.248168
min,1.000000,2016.0,1900.000000,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,1.000000,0.000000e+00,0.000000e+00,-1.154170e+05,0.000000e+00,-0.800000,-0.020000
25%,609.500000,2016.0,1929.000000,1.000000,1.000000,2.872800e+04,0.000000,2.768500e+04,44.000000,1.265929e+06,0.000000e+00,7.075250e+05,0.000000e+00,19.335000,0.340000
50%,21213.000000,2016.0,1965.000000,1.000000,2.000000,4.721500e+04,0.000000,4.508200e+04,71.000000,2.739122e+06,0.000000e+00,1.633105e+06,4.313920e+05,48.350000,0.850000
75%,24629.500000,2016.0,1988.000000,1.000000,4.000000,1.028880e+05,0.000000,9.290600e+04,89.000000,7.227405e+06,0.000000e+00,5.046944e+06,1.409886e+06,136.735000,1.940000
max,50226.000000,2016.0,2015.000000,9.000000,99.000000,1.952220e+06,512608.000000,1.765970e+06,100.000000,2.966717e+08,1.349435e+08,2.745325e+08,1.381912e+08,12307.160000,25.710000


In [490]:
data_length_before = data.shape[0]
cols = data.select_dtypes(include=[np.number]).columns
data = data[~(data[cols] < 0).any(axis=1)].copy()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1


In [491]:
data_length_before = data.shape[0]
difference = abs(data['PropertyGFATotal'] - (data['PropertyGFABuilding(s)'] + data['PropertyGFAParking']))
data = data[difference == 0]
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  0


In [492]:
#J'enregistre les data des EnergyScore rempli à part pour plus tard
data_energy_star_filtered = data[data['ENERGYSTARScore'].notna()]
data_energy_star_filtered.to_csv('data_energy_star_filtered.csv', index=False)

data.to_csv('data_cleaned.csv', index=False)
